In [ ]:
import random
import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.models.rnn import rnn_cell
from tensorflow.models.rnn import seq2seq
from tensorflow.models.rnn.translate import data_utils


class DateNormModel(object):
   
    def __init__(self, num_source_labels, num_target_labels, hidden_units,
                 num_layers, max_gradient_norm, batch_size, learning_rate,
                 learning_rate_decay_factor, use_lstm=False, forward_only=False):
        """Create the model.
        Args:
          num_source_labels: size of the source vocabulary.
          num_target_labels: size of the target vocabulary.
          hidden_units: number of units in each layer of the model.
          num_layers: number of layers in the model.
          max_gradient_norm: gradients will be clipped to maximally this norm.
          batch_size: the size of the batches used during training;
            the model construction is independent of batch_size, so it can be
            changed after initialization if this is convenient, e.g., for decoding.
          learning_rate: learning rate to start with.
          learning_rate_decay_factor: decay learning rate by this much when needed.
          use_lstm: if true, we use LSTM cells instead of GRU cells.
          num_samples: number of samples for sampled softmax.
          forward_only: if set, we do not construct the backward pass in the model.
        """
        self.source_vocab_size = source_vocab_size
        self.target_vocab_size = target_vocab_size
        self.buckets = buckets
        self.batch_size = batch_size
        self.learning_rate = tf.Variable(float(learning_rate), trainable=False)
        self.learning_rate_decay_op = self.learning_rate.assign(
            self.learning_rate * learning_rate_decay_factor)
        self.global_step = tf.Variable(0, trainable=False)
        
        
        single_cell = rnn_cell.BasicLSTMCell(size)
        if num_layers > 1:
            cell = rnn_cell.MultiRNNCell([single_cell] * num_layers)
            
        # The seq2seq function: we use embedding for the input and attention.
        def seq2seq_f(encoder_inputs, decoder_inputs, do_decode):
            return seq2seq.basic_rnn_seq2seq(encoder_inputs, decoder_inputs, cell)
        
        # Feeds for inputs.
        self.encoder_inputs = []
        self.decoder_inputs = []
        self.target_weights = []
        for i in xrange(buckets[-1][0]):  # Last bucket is the biggest one.
            self.encoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                      name="encoder{0}".format(i)))
        for i in xrange(buckets[-1][1] + 1):
            self.decoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                      name="decoder{0}".format(i)))
            
        # Our targets are decoder inputs shifted by one.
        targets = [self.decoder_inputs[i + 1]
                   for i in xrange(len(self.decoder_inputs) - 1)]
        
        # Training outputs and losses.
        if forward_only:
            self.outputs, self.losses = seq2seq.model_with_buckets(
                self.encoder_inputs, self.decoder_inputs, targets,
                self.target_weights, buckets, self.target_vocab_size,
                lambda x, y: seq2seq_f(x, y, True),
                softmax_loss_function=softmax_loss_function)
            # If we use output projection, we need to project outputs for decoding.
            if output_projection is not None:
                for b in xrange(len(buckets)):
                    self.outputs[b] = [tf.matmul(output, output_projection[0]) +
                                       output_projection[1]
                                       for output in self.outputs[b]]
        else:
            self.outputs, self.losses = seq2seq.model_with_buckets(
                self.encoder_inputs, self.decoder_inputs, targets,
                self.target_weights, buckets, self.target_vocab_size,
                lambda x, y: seq2seq_f(x, y, False),
                softmax_loss_function=softmax_loss_function)
        # Gradients and SGD update operation for training the model.
        params = tf.trainable_variables()
        if not forward_only:
            self.gradient_norms = []
            self.updates = []
            opt = tf.train.GradientDescentOptimizer(self.learning_rate)
            for b in xrange(len(buckets)):
                gradients = tf.gradients(self.losses[b], params)
                clipped_gradients, norm = tf.clip_by_global_norm(gradients,
                                                                 max_gradient_norm)
                self.gradient_norms.append(norm)
                self.updates.append(opt.apply_gradients(
                    zip(clipped_gradients, params), global_step=self.global_step))
        self.saver = tf.train.Saver(tf.all_variables())

    def step(self, session, encoder_inputs, decoder_inputs, target_weights,
             bucket_id, forward_only):
        """Run a step of the model feeding the given inputs.
        Args:
          session: tensorflow session to use.
          encoder_inputs: list of numpy int vectors to feed as encoder inputs.
          decoder_inputs: list of numpy int vectors to feed as decoder inputs.
          target_weights: list of numpy float vectors to feed as target weights.
          bucket_id: which bucket of the model to use.
          forward_only: whether to do the backward step or only forward.
        Returns:
          A triple consisting of gradient norm (or None if we did not do backward),
          average perplexity, and the outputs.
        Raises:
          ValueError: if length of encoder_inputs, decoder_inputs, or
            target_weights disagrees with bucket size for the specified bucket_id.
        """
        # Check if the sizes match.
        encoder_size, decoder_size = self.buckets[bucket_id]
        if len(encoder_inputs) != encoder_size:
            raise ValueError("Encoder length must be equal to the one in bucket,"
                             " %d != %d." % (len(encoder_inputs), encoder_size))
        if len(decoder_inputs) != decoder_size:
            raise ValueError("Decoder length must be equal to the one in bucket,"
                             " %d != %d." % (len(decoder_inputs), decoder_size))
        if len(target_weights) != decoder_size:
            raise ValueError("Weights length must be equal to the one in bucket,"
                             " %d != %d." % (len(target_weights), decoder_size))
        # Input feed: encoder inputs, decoder inputs, target_weights, as provided.
        input_feed = {}
        for l in xrange(encoder_size):
            input_feed[self.encoder_inputs[l].name] = encoder_inputs[l]
        for l in xrange(decoder_size):
            input_feed[self.decoder_inputs[l].name] = decoder_inputs[l]
            input_feed[self.target_weights[l].name] = target_weights[l]
        # Since our targets are decoder inputs shifted by one, we need one more.
        last_target = self.decoder_inputs[decoder_size].name
        input_feed[last_target] = np.zeros([self.batch_size], dtype=np.int32)
        # Output feed: depends on whether we do a backward step or not.
        if not forward_only:
            output_feed = [self.updates[bucket_id],  # Update Op that does SGD.
                           self.gradient_norms[bucket_id],  # Gradient norm.
                           self.losses[bucket_id]]  # Loss for this batch.
        else:
            output_feed = [self.losses[bucket_id]]  # Loss for this batch.
            for l in xrange(decoder_size):  # Output logits.
                output_feed.append(self.outputs[bucket_id][l])
        outputs = session.run(output_feed, input_feed)
        if not forward_only:
            return outputs[1], outputs[2], None  # Gradient norm, loss, no outputs.
        else:
            return None, outputs[0], outputs[1:]  # No gradient norm, loss, outputs.

    


In [ ]:
run()

In [ ]:
raw_data = reader.ptb_raw_data(FLAGS.data_path)
train_data, valid_data, test_data, _ = raw_data

for step, (x, y) in enumerate(reader.ptb_iterator(train_data,1 3)):
    print(step, x, y)

'February 21, 2016'

In [114]:
from datetime import datetime
import random

from random import randrange
from datetime import timedelta

START_DATE = datetime.strptime('1950-01-01', '%Y-%m-%d')
END_DATE = datetime.strptime('2050-12-31', '%Y-%m-%d')
FORMAT_TOKENS = ('%a', '%A', '%d', '%b', '%B', '%m', '%y', '%Y')

def random_date(start=START_DATE, end=END_DATE):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

def random_date_string(date):
    random_format_tokens = random.sample(FORMAT_TOKENS, 5)
    date_format = '-'.join(random_format_tokens)
    print(date_format)
    return date.strftime(date_format)
    #print("\{\:{}\}".format(date_format))

date = random_date()
date_string = random_date_string(date)
    
print(date_string, "->", date.strftime('%Y-%m-%d')) 

%d-%A-%B-%b-%a
03-Saturday-August-Aug-Sat -> 2047-08-03


In [111]:
import tensorflow as tf
from date_norm_model import DateNormModel, train
from date_generator import INPUT_SYMBOL_TO_IDX, INPUT_SEQ_LEN, OUTPUT_SYMBOL_TO_IDX, OUTPUT_SEQ_LEN
from date_generator import generate_training_data, OUTPUT_SYMBOLS
from date_norm_model import decode_output_sequences
    
hidden_units = 128
batch_size = 8

with tf.Graph().as_default(), tf.Session() as session:
    
    with tf.variable_scope("model", reuse=None):
        training_model = DateNormModel(session=session,
                                        hidden_units = hidden_units, 
                                        input_sequence_len = INPUT_SEQ_LEN,
                                        output_sequence_len = OUTPUT_SEQ_LEN,
                                        num_input_symbols = len(INPUT_SYMBOL_TO_IDX),
                                        num_output_symbols = len(OUTPUT_SYMBOL_TO_IDX),
                                        batch_size = batch_size,
                                        is_training=True)
        
    with tf.variable_scope("model", reuse=True):
        testing_model = DateNormModel(session=session,
                                        hidden_units = hidden_units, 
                                        input_sequence_len = INPUT_SEQ_LEN,
                                        output_sequence_len = OUTPUT_SEQ_LEN,
                                        num_input_symbols = len(INPUT_SYMBOL_TO_IDX),
                                        num_output_symbols = len(OUTPUT_SYMBOL_TO_IDX),
                                        batch_size = batch_size,
                                        is_training=False)
        
    
    train(model=training_model, 
          batch_size=batch_size,
          num_epochs=10,
          batches_per_epoch=32)
    

    data_iterator = generate_training_data(batch_size=batch_size)

    x, y = next(data_iterator)
    #print(batch_size, len(x), len(INPUT_SYMBOL_TO_IDX))
    target_strings = decode_output_sequences(y, symbols=OUTPUT_SYMBOLS)
    symbol_probs = testing_model.predict(x)
    pred_strings = decode_output_sequences(symbol_probs, symbols=OUTPUT_SYMBOLS)
    print(target_strings)
    print(pred_strings)
    print(list(zip(target_strings, pred_strings)))
    

31/32 (epoch 0), train_loss = 1.858, time/batch = 0.023
31/32 (epoch 1), train_loss = 1.594, time/batch = 0.023
31/32 (epoch 2), train_loss = 1.262, time/batch = 0.022
31/32 (epoch 3), train_loss = 1.159, time/batch = 0.023
31/32 (epoch 4), train_loss = 1.089, time/batch = 0.039
31/32 (epoch 5), train_loss = 1.147, time/batch = 0.023
31/32 (epoch 6), train_loss = 1.131, time/batch = 0.024
31/32 (epoch 7), train_loss = 1.006, time/batch = 0.024
31/32 (epoch 8), train_loss = 0.937, time/batch = 0.023
31/32 (epoch 9), train_loss = 0.908, time/batch = 0.023
encoder_inputs (8, 8, 43)
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
1 (8, 11)
['GO19830831', 'GO19800823'

RuntimeError: Attempted to use a closed Session.

In [105]:
iterator = generate_training_data(batch_size=1)

In [106]:
x, y = next(iterator)
print(x.shape, y.shape)
print(x)

(1, 8, 43) (1, 9, 11)
[[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

In [107]:
from date_generator import generate_training_data, OUTPUT_SYMBOLS, INPUT_SYMBOLS
input_strings = decode_output_sequences(x, symbols=INPUT_SYMBOLS)
print(input_strings)
target_strings = decode_output_sequences(y, symbols=OUTPUT_SYMBOLS)
print(target_strings)

['41110402']
['GO20401114']


In [134]:
dense_to_one_hot(np.array([[0, 1, 2, 2], 
                           [1, 1, 2, 2]]), num_classes=4)

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]]])

In [131]:
dense_to_one_hot(np.array([1, 1, 2, 2]), num_classes=3)

array([[ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.]])

In [144]:
len(max(['asd', 'asdsd'], key=len))

5

In [6]:
y, x = next(generate_training_data())

In [7]:
print(y, x)

[[[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 ..., 
 [[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  1. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  